In [689]:
import json
import requests
from lxml import etree
import time
import sqlite3
from datetime import date

In [457]:
BASE_URL = 'https://www.wta.org/go-hiking/hikes/' # base + <id> for individual page

In [642]:
def retrieve_hike_urls(): 
    HIKE_LIST_URL = 'https://www.wta.org/go-hiking/@@trailhead-text-search?jsonp_callback=&query=&start=0&num=9999&_=1629843008980'
    response = requests.get(HIKE_LIST_URL)
    data = json.loads(response.text[1:-1])
    del data['start']
    for hike in data['data']:
        del hike['m']
        hike['url'] = BASE_URL + hike['id']
    return data

In [639]:
def retrieve_and_save_hike_urls(filename):
    data = retrieve_hike_urls()
    with open(filename, 'w+') as file:
        file.write(json.dumps(data))

In [636]:
def load_hike_urls(filename):
    with open(filename) as file:
        return json.loads(file.read())

In [635]:
def retrieve_hike_html(hike_id):
    time.sleep(1)
    response = requests.get(BASE_URL + hike_id)
    return response.text

In [684]:
def save_all_hikes(filename):
    hikes = []
    all_hike_urls = load_hike_urls('wta_trail_list.json')['data']
    for index, hike in enumerate(all_hike_urls):
        print(str(index+1) + ' out of ' + str(len(all_hike_urls)))
        print('(' + hike['name'] + ')')
        hikes.append(extract_details(hike['id']))
    with open(filename, 'w+') as file:
        file.write(json.dumps(hikes))

In [686]:
def extract_details(hike_id):
    hike_html = etree.HTML(retrieve_hike_html(hike_id))
    
    details = {
        'trail_id' : hike_id
    }
    
    try:
        alerts = []
        for alert in hike_html.xpath('//div[@id="hike-top"]/div[@class="alerts-and-conditions"]/div'):
            if 'red' in alert.attrib['class']:      
                alerts.append({
                    'type' : 'red',
                    'text' : alert.getchildren()[0].text
                })
            else:            
                alerts.append({
                    'type' : 'parking',
                    'text' : alert.getchildren()[1].text
                })

        details['alerts'] = alerts
    except IndexError:
        pass
    
    try:
        details['last_maintained'] = hike_html.xpath('//div[@class="last-maintained"]/div')[0].text
    except IndexError:
        pass
    
    try:
        details['name'] = hike_html.xpath('//div[@id="hike-top"]/h1')[0].text
    except IndexError:
        pass
        
    try:
        features = []
        for feature in hike_html.xpath('//div[@id="hike-features"]/div'):
            try: 
                features.append(feature.attrib['data-title'])
            except KeyError:
                pass
        details['features'] = features
    except IndexError:
        pass
    
    try: 
        details['distance'] = hike_html.xpath('//div[@id="distance"]/span')[0].text
    except IndexError:
        pass
        
    try:
        elevation = hike_html.xpath('//div[@class="hike-stat"]/div/span')
        details['elevation'] = {
            'gain' : elevation[0].text,
            'highest_point' : elevation[1].text
        }
    except IndexError:
        pass
        
    try:
        details['rating'] = {
            'stars' : hike_html.xpath('//div[@class="current-rating"]')[0].text,
            'num_votes' : hike_html.xpath('//div[@class="rating-count"]')[0].text.strip()[1:].split(' ')[0]
        }
    except IndexError:
        pass
        
    try:
        details['location'] = hike_html.xpath('//div[@id="hike-stats"]/div[@class="hike-stat"]/div')[0].text
    except IndexError:
        pass
        
    try: 
        details['trailhead_coordinates'] = {
            'latitude' : hike_html.xpath('//div[@class="latlong"]/span')[0].text,
            'longitude' : hike_html.xpath('//div[@class="latlong"]/span')[1].text
        }
    except IndexError:
        pass
        
    try:
        details['wta_author'] = hike_html.xpath('//div[@class="img-text-grouping"]/p/a/span')[0].text
    except IndexError:
        pass
    
    try:
        details['driving_directions'] = hike_html.xpath('//div[@id="driving-directions"]/p')[1].text
    except IndexError:
        pass
        
    try:
        details['hike_description'] = '\n'.join(hike_html.xpath('//div[@id="hike-body-text"]/p/text()'))
    except IndexError:
        pass

    URL = BASE_URL + hike_id + '/@@related_tripreport_listing'
    
    def get_all_trs():
        current_page = get_tr_page(URL)
        trs = []
        while current_page:
            current_page = get_tr_page(current_page['next_page_url'])
            if current_page:
                trs += current_page['tr_urls']
        return trs
    
    def get_tr_page(url):
        time.sleep(1)
        
        html = etree.HTML(requests.get(url).text)
        
        try:
            tr_urls = [tr.attrib['href'] for tr in html.xpath('//a[@class="show-with-full full-report-link visualNoPrint hidden-480 wta-action button"]')]
            return {
                'next_page_url' : html.xpath('//li[@class="next"]/a')[0].attrib['href'],
                'tr_urls' : tr_urls,
            }
        except IndexError:
            return None
        
    def get_tr(url):
        time.sleep(1)
        
        html = etree.HTML(requests.get(url).text)
        
        tr = {}
        
        try:
            tr['author'] = html.xpath('string(//span[@itemprop="author"]/a)').strip()
        except IndexError:
            pass
        
        try:
            condition_elements = html.xpath('//div[@class="trip-condition"]')
            tr['conditions'] = [{
                c.getchildren()[0].text.lower().replace(' ', '_') : c.getchildren()[1].text
            } for c in condition_elements]
        except IndexError:
            pass
        
        try:
            tr['likes'] = html.xpath('//span[@class="tally-total"]/text()')[0]
        except IndexError:
            pass
        
        try:
            tr['report'] = '\n'.join(html.xpath('//div[@id="tripreport-body-text"]/p/text()'))
        except IndexError:
            pass
        
        try:
            tr['date_hiked'] = html.xpath('//span[@class="elapsed-time"]/text()')[0]
        except IndexError:
            pass
        
        return tr
    
    # get trip reports
#     trs = get_all_trs()
#     print('getting ' + str(len(trs)) + ' TRs...')
#     details['trip_reports'] = [get_tr(url) for url in get_all_trs()]
    print()
    
    return details

In [687]:
hikes = load_hikes('wta_trail_list.json')
save_all_hikes('wta_hikes.json')

1 out of 3897
(45 Mile Sheep Drive (Central Cascades -- Entiat Mountains/Lake Chelan))

2 out of 3897
(76 Creek (North Cascades -- Mountain Loop Highway))

3 out of 3897
(360 Trails (Puget Sound and Islands -- Seattle-Tacoma Area))

4 out of 3897
(520 Trail (Puget Sound and Islands -- Seattle-Tacoma Area))

5 out of 3897
(2773' 353'P (Central Cascades))

6 out of 3897
(A Quiet Place Park (Puget Sound and Islands -- Seattle-Tacoma Area))

7 out of 3897
(Aasgard Pass (Central Cascades -- Leavenworth Area))

8 out of 3897
(Abercrombie Mountain Trail (Eastern Washington -- Selkirk Range))

9 out of 3897
(Abernathy Pass via Cedar Creek (North Cascades -- North Cascades Highway - Hwy 20))

10 out of 3897
(Abernathy Peak (North Cascades -- Methow/Sawtooth))

11 out of 3897
(Abiel Pass (Snoqualmie Region))

12 out of 3897
(Abiel Peak (Snoqualmie Region -- Snoqualmie Pass))

13 out of 3897
(Adams Creek Meadows (South Cascades -- Mount Adams Area))

14 out of 3897
(Admiralty Inlet Natural Area P


104 out of 3897
(Augspurger Mountain - North (Southwest Washington -- Columbia River Gorge - WA))

105 out of 3897
(Augspurger Trail (Southwest Washington -- Columbia River Gorge - WA))

106 out of 3897
(Aurora Creek (Olympic Peninsula -- Northern Coast))

107 out of 3897
(Aurora Divide (Olympic Peninsula -- Northern Coast))

108 out of 3897
(Aurora Ridge (Olympic Peninsula -- Northern Coast))

109 out of 3897
(Austera Peak (North Cascades -- North Cascades Highway - Hwy 20))

110 out of 3897
(Avalanche Mountain (Snoqualmie Region -- Snoqualmie Pass))

111 out of 3897
(Axis Peak  (Central Cascades -- Leavenworth Area))

112 out of 3897
(Bachelor Meadows (Central Cascades))

113 out of 3897
(Backbone Ridge (South Cascades -- White Pass/Cowlitz River Valley))

114 out of 3897
(Backpacker Camp (Central Cascades -- Entiat Mountains/Lake Chelan))

115 out of 3897
(Badger Mountain - Canyon Trail (Central Washington -- Tri-Cities))

116 out of 3897
(Badger Mountain - Sagebrush Trail (Central


209 out of 3897
(Beefhide Butte (North Cascades -- Methow/Sawtooth))

210 out of 3897
(Beezley Hills Preserve (Central Washington -- Grand Coulee))

211 out of 3897
(Behren's Woods (Southwest Washington -- Vancouver Area))

212 out of 3897
(Bell Pass (North Cascades -- Mount Baker Area))

213 out of 3897
(Bell Point and English Camp (Puget Sound and Islands -- San Juan Islands))

214 out of 3897
(Bellevue Botanical Garden (Puget Sound and Islands -- Seattle-Tacoma Area))

215 out of 3897
(Bells Mountain (Southwest Washington -- Columbia River Gorge - WA))

216 out of 3897
(Bench and Snow Lakes (Mount Rainier Area -- SE - Cayuse Pass/Stevens Canyon))

217 out of 3897
(Bench Lake Loop (South Cascades -- Mount Adams Area))

218 out of 3897
(Bennington Lake (Eastern Washington -- Palouse and Blue Mountains))

219 out of 3897
(Berkeley Park (Mount Rainier Area -- NE - Sunrise/White River))

220 out of 3897
(Bernhardt Mine (North Cascades -- Pasayten))

221 out of 3897
(Bernice Lake (Centra


321 out of 3897
(Boardman Lake (North Cascades -- Mountain Loop Highway))

322 out of 3897
(Boardman Lake Snowshoe (North Cascades -- Mountain Loop Highway))

323 out of 3897
(Bob Heirman Wildlife Preserve (Puget Sound and Islands -- Seattle-Tacoma Area))

324 out of 3897
(Bobs Lakes (Central Washington -- Potholes Region))

325 out of 3897
(Bogachiel Peak (Olympic Peninsula -- Northern Coast))

326 out of 3897
(Bogachiel River (Olympic Peninsula -- Pacific Coast))

327 out of 3897
(Boiling Lake (North Cascades -- Methow/Sawtooth))

328 out of 3897
(Bonanza Peak (Central Cascades -- Entiat Mountains/Lake Chelan))

329 out of 3897
(Bonaparte Trail (Eastern Washington -- Okanogan Highlands/Kettle River Range))

330 out of 3897
(Bone Lake (South Cascades))

331 out of 3897
(Bonneville Trail (Issaquah Alps -- Tiger Mountain))

332 out of 3897
(Bootjack Mountain (Central Cascades -- Leavenworth Area))

333 out of 3897
(Bootleg Trail (Issaquah Alps -- Tiger Mountain))

334 out of 3897
(Bord


427 out of 3897
(Butte Camp (South Cascades -- Mount St. Helens))

428 out of 3897
(Butte Creek (Central Cascades -- Entiat Mountains/Lake Chelan))

429 out of 3897
(Butte Creek Sitka Spruce Grove (Southwest Washington -- Long Beach Area))

430 out of 3897
(Buttermilk Ridge (North Cascades -- Methow/Sawtooth))

431 out of 3897
(By Pass Trail (Issaquah Alps -- Cougar Mountain))

432 out of 3897
(Bygone Byways Interpretive Trail (Central Cascades -- Stevens Pass - East))

433 out of 3897
(BZ Trail (Southwest Washington -- Columbia River Gorge - WA))

434 out of 3897
(Cabin Creek Snowshoe (Snoqualmie Region -- Snoqualmie Pass))

435 out of 3897
(Cabin Trail (Eastern Washington -- Okanogan Highlands/Kettle River Range))

436 out of 3897
(Cabinet Creek (North Cascades))

437 out of 3897
(Cabinet Peak (North Cascades -- North Cascades Highway - Hwy 20))

438 out of 3897
(Cable Line Trail (Issaquah Alps -- Tiger Mountain))

439 out of 3897
(Cadet Peak (North Cascades -- Mountain Loop Highway

...


3424 out of 3897
(Tammi Wilson Memorial Trail (Puget Sound and Islands -- Bellingham Area))

3425 out of 3897
(Taneum Lake (Snoqualmie Region -- Cle Elum Area))

3426 out of 3897
(Taneum Ridge (Snoqualmie Region -- Cle Elum Area))

3427 out of 3897
(Tank Lakes (Central Cascades -- Stevens Pass - West))

3428 out of 3897
(Tanner Landing Park (Snoqualmie Region -- North Bend Area))

3429 out of 3897
(Tarbell Trail (Southwest Washington -- Lewis River Region))

3430 out of 3897
(Tatie Peak and Grasshopper Pass (North Cascades -- Pasayten))

3431 out of 3897
(Tatoosh Buttes (North Cascades))

3432 out of 3897
(Tatoosh Lakes (South Cascades -- White Pass/Cowlitz River Valley))

3433 out of 3897
(Tatoosh Lookout (South Cascades -- White Pass/Cowlitz River Valley))

3434 out of 3897
(Tatoosh Ridge (South Cascades -- White Pass/Cowlitz River Valley))

3435 out of 3897
(Taylor Mountain (Issaquah Alps))

3436 out of 3897
(Taylor Mountain - Boot Trail (Issaquah Alps -- Taylor Mountain))

3437 ou


3528 out of 3897
(Timothy Lake Loop (Southwest Washington -- Columbia River Gorge - OR))

3529 out of 3897
(Tin Cup Joe Falls (Snoqualmie Region -- North Bend Area))

3530 out of 3897
(Tinkham Discovery (Snoqualmie Region -- Snoqualmie Pass))

3531 out of 3897
(Tiptop (Central Cascades -- Blewett Pass))

3532 out of 3897
(Tired Creek (Snoqualmie Region -- Salmon La Sac/Teanaway))

3533 out of 3897
(Titlow Park (Puget Sound and Islands -- Seattle-Tacoma Area))

3534 out of 3897
(To Know a Tree (North Cascades -- North Cascades Highway - Hwy 20))

3535 out of 3897
(Tolmie Peak Lookout - Eunice Lake (Mount Rainier Area -- NW - Carbon River/Mowich))

3536 out of 3897
(Tolmie State Park - Four Cedars Trail (Puget Sound and Islands -- Seattle-Tacoma Area))

3537 out of 3897
(Tolt Pipeline Trail (Puget Sound and Islands -- Seattle-Tacoma Area))

3538 out of 3897
(Tolt-MacDonald Park (Puget Sound and Islands -- Seattle-Tacoma Area))

3539 out of 3897
(Tom Dick & Harry Lakes (Central Cascades 


3630 out of 3897
(Two Dollar Trail (Puget Sound and Islands -- Bellingham Area))

3631 out of 3897
(Two Lakes (South Cascades))

3632 out of 3897
(Two Little Lakes (Central Cascades))

3633 out of 3897
(Tyee Lookout (Central Cascades -- Entiat Mountains/Lake Chelan))

3634 out of 3897
(Tyler Peak Trail (Olympic Peninsula -- Hood Canal))

3635 out of 3897
(Ueland Tree Farm (Olympic Peninsula -- Kitsap Peninsula))

3636 out of 3897
(Ugly Duckling 5140' (Snoqualmie Region -- Snoqualmie Pass))

3637 out of 3897
(Ulalach Peak (North Cascades -- Mountain Loop Highway))

3638 out of 3897
(Umatilla Rim Trail (Eastern Washington -- Palouse and Blue Mountains))

3639 out of 3897
(Umtanum Creek Canyon (Central Washington -- Yakima))

3640 out of 3897
(Umtanum Creek Falls (Central Washington -- Yakima))

3641 out of 3897
(Umtanum Ridge Crest (Central Washington -- Yakima))

3642 out of 3897
(Unicorn Peak (Mount Rainier Area -- SE - Cayuse Pass/Stevens Canyon))

3643 out of 3897
(Union Bay Natural


3735 out of 3897
(Wenatchee (Eastern Washington))

3736 out of 3897
(Wenatchee Confluence State Park - Horan Natural Area (Central Washington -- Wenatchee))

3737 out of 3897
(Wenatchee Crest Snowshoe (Blewett Pass) (Central Cascades -- Blewett Pass))

3738 out of 3897
(Wenatchee Guard Station (Eastern Washington -- Palouse and Blue Mountains))

3739 out of 3897
(Wenatchee Ridge Snowshoe (Lake Wenatchee Area) (Central Cascades -- Stevens Pass - East))

3740 out of 3897
(Wenatchee River Road (Central Cascades -- Leavenworth Area))

3741 out of 3897
(Weowna Park (Puget Sound and Islands -- Seattle-Tacoma Area))

3742 out of 3897
(West Access Trail (Issaquah Alps -- Squak Mountain))

3743 out of 3897
(West Bank (North Cascades))

3744 out of 3897
(West Butte (Eastern Washington -- Palouse and Blue Mountains))

3745 out of 3897
(West Cady Ridge to Benchmark Mountain (Central Cascades -- Stevens Pass - West))

3746 out of 3897
(West Craggy Peak (North Cascades -- Pasayten))

3747 out of 38


3833 out of 3897
(William O. Douglas Wilderness (Central Washington -- Yakima))

3834 out of 3897
(Williams Creek (North Cascades -- Methow/Sawtooth))

3835 out of 3897
(Williams Lake (Snoqualmie Region))

3836 out of 3897
(Willow Creek (North Cascades))

3837 out of 3897
(Willow Lake Trail (North Cascades -- North Cascades Highway - Hwy 20))

3838 out of 3897
(Wilson Creek (Snoqualmie Region -- Cle Elum Area))

3839 out of 3897
(Wilson Creek (Central Cascades -- Entiat Mountains/Lake Chelan))

3840 out of 3897
(Wilson Creek (Central Washington -- Grand Coulee))

3841 out of 3897
(Winchester Mountain (North Cascades -- Mount Baker Area))

3842 out of 3897
(Winchester Wasteway (Central Washington -- Potholes Region))

3843 out of 3897
(Wind Mountain (Southwest Washington -- Columbia River Gorge - WA))

3844 out of 3897
(Windy Creek (Eastern Washington))

3845 out of 3897
(Windy Lake (Snoqualmie Region))

3846 out of 3897
(Windy Pass (Central Cascades -- Leavenworth Area))

3847 out of 

In [746]:
def file_to_db(filename):
    with open(filename) as file:
        data = json.loads(file.read())
        with sqlite3.connect('hikes.db') as con:
            cursor = con.cursor()
            for hike in data:
                cursor.execute('INSERT INTO hike (slug, last_maintained, name, distance, gain, highest_point, stars, num_votes, th_lat, th_long, wta_author, driving_directions, hike_description, last_scraped) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
                    [hike.get('trail_id'), hike.get('last_maintained'), hike.get('name'), hike.get('distance'), hike.get('elevation', {}).get('gain'), hike.get('elevation', {}).get('highest_point'), hike.get('rating', {}).get('stars'), hike.get('rating', {}).get('num_votes'), hike.get('trailhead_coordinates', {}).get('latitude'), hike.get('trailhead_coordinates', {}).get('longitude'), hike.get('wta_author'), hike.get('driving_directions'), hike.get('hike_description'), date.today().strftime("%d/%m/%Y")])
                hike_id = cursor.lastrowid
                if 'features' in hike:
                    for feature in hike['features']:
                        cursor.execute('INSERT INTO feature (hike_id, type) VALUES (?, ?)', [hike_id, feature])
                if 'alerts' in hike:
                    for alert in hike['alerts']:
                        cursor.execute('INSERT INTO alert (hike_id, type, text) VALUES (?, ?, ?)', [hike_id, alert['type'], alert['text']])

In [748]:
file_to_db('wta_hikes.json')

In [766]:
def washed_out_roads_geojson():
    with sqlite3.connect('hikes.db') as con:
        cursor = con.cursor()
        results = cursor.execute('SELECT hike.name, alert.text, hike.th_lat, hike.th_long FROM hike INNER JOIN alert ON alert.hike_id = hike.id WHERE alert.type = "red" AND alert.text LIKE "%road%"').fetchall()
        return {
            'type' : 'FeatureCollection',
            'features' : [{
                'type' : 'Feature',
                'geometry' : {
                    'type' : 'Point',
                    'coordinates' : [result[3], result[2]]
                },
                'properties' : {
                    'title' : result[0],
                    'description' : result[1]
                }
                } for result in results]
        }

In [767]:
with open('washed_out_roads.json', 'w+') as file:
    file.write(json.dumps(washed_out_roads_geojson()))